# Example for creating a custom process in AeroMAPS

This document aims to show the recommended way to create a custom process (configuration file and custom models) and execute them within AeroMAPS.

## Load and process

First, the user has to load the framework and generate a process.

In [ ]:
%matplotlib widget
from aeromaps import create_process
from aeromaps.core.models import (
    models_traffic,
    models_efficiency_top_down,
    models_energy_without_fuel_effect,
    models_offset,
    models_climate_simple_gwpstar,
    models_sustainability,
)

## Create a custom model

Here by default we import several aggregated reference models, such as `models_traffic`, that contains basic models used in AeroMAPS. Overall, these different aggregated models include 100 basic models. We recommend to keep these default models to not remove any outputs that may be needed by other models.
For instance, the `models_traffic` is a dictionnary structure with keys that are the name of the model and the value an instance of the model:

```python
models_traffic = {
    "rpk_measures": RPKMeasures("rpk_measures"),
    "rpk": RPK("rpk"),
    "rpk_reference": RPKReference("rpk_reference"),
    "total_aircraft_distance": TotalAircraftDistance("total_aircraft_distance"),
    "rtk": RTK("rtk"),
    "rtk_reference": RTKReference("rtk_reference"),
    "ask": ASK("ask")
}

```

We create a custom model example `MeanDistancePerInhabitantFlyer` that is located [here](./models/mean_distance_per_inhabitant_and_flyer.py), in the file `models/mean_distance_per_inhabitant_and_flyer.py`. To integrate it in to the AeroMAPS process we propose to import it and to add it to the dictionnary containing the models.

In [ ]:
from models.mean_distance_per_inhabitant_and_flyer import MeanDistancePerInhabitantFlyer

extended_models = {
    "models_traffic": models_traffic,
    "models_efficiency_top_down": models_efficiency_top_down,
    "models_energy_without_fuel_effect": models_energy_without_fuel_effect,
    "models_offset": models_offset,
    "models_climate_simple_gwpstar": models_climate_simple_gwpstar,
    "models_sustainability": models_sustainability,
    "mean_distance_per_inhabitant_and_flyer": MeanDistancePerInhabitantFlyer(
        "mean_distance_per_inhabitant_and_flyer"
    ),
}

This new ensemble of models can be provided to the process.

## Create the process

Here we show how we can use a configuration file with an example [here](./data/config.json), in the file `data/config.json`.
There you can specify the relative path to the different files.
For example, you can provide a custom input file with the parameters you want to modify with an example [here](./data/inputs.json), in the file `data/inputs.json`.

Be careful, do not forget to set the inputs required by your models, either in the `resources/data/parameters.json` file if you modify the source code, or directly in the `inputs.json` file from your configuration file. You can also change the inputs data directly through the code after the creation of the process (see below).

In [ ]:
process = create_process(configuration_file="data/config.json", models=extended_models)

In [ ]:
process.parameters.world_inhabitant_number_reference_years = [2020, 2030, 2040, 2050]
process.parameters.world_inhabitant_number_reference_years_values = [
    7.805e9,
    8.512e9,
    9.159e9,
    9.687e9,
]  # Assumption based on the 2022 Revision of World Population Prospects from the United Nations
process.parameters.inhabitant_flyer_share_reference_years = [2020, 2030, 2040, 2050]
process.parameters.inhabitant_flyer_share_reference_years_values = [
    11,
    14,
    17,
    20,
]  # Assumption based on a linear increase from Gossling and Humpe (2020) value

## Compute

Once all the parameters have been set up, the user can compute.

In [ ]:
process.compute()

## Results

The user can then display the results. The user has access to float outputs but also to annual data outputs, with the possibility of choosing the output.

In [ ]:
process.data["vector_outputs"][["mean_distance_per_inhabitant", "mean_distance_per_flyer"]]